In [1]:
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/functions.py

!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/assets.txt
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/threats.txt

--2021-04-11 19:00:52--  https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6897 (6.7K) [text/plain]
Saving to: ‘functions.py’

functions.py        100%[===================>]   6.74K  --.-KB/s    in 0s      

2021-04-11 19:00:52 (80.2 MB/s) - ‘functions.py’ saved [6897/6897]

--2021-04-11 19:00:52--  https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/assets.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3953 (3

In [2]:


import functions
from functions import *
import spacy
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

!wget https://raw.githubusercontent.com/tylerneylon/explacy/master/explacy.py



--2021-04-11 19:00:56--  https://raw.githubusercontent.com/tylerneylon/explacy/master/explacy.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6896 (6.7K) [text/plain]
Saving to: ‘explacy.py’

explacy.py          100%[===================>]   6.73K  --.-KB/s    in 0s      

2021-04-11 19:00:56 (32.4 MB/s) - ‘explacy.py’ saved [6896/6896]



In [3]:
import explacy

sentence = 'Computers are exposed to a Device Lost Threat with prob 5.0 and impact 1.0 and to a Device Theft Threat with prob 4.0 and impact 2.0.'
sentence = 'Computers are exposed to a Device Lost Threat with prob 5.0 and impact 1.0'
             
nlp = spacy.load('en')

# explacy.print_parse_info(nlp, sentences)


explacy.print_parse_info(nlp, sentence)

Dep tree       Token     Dep type  Lemma    Part of Sp
────────────── ───────── ───────── ──────── ──────────
          ┌──► Computers nsubjpass computer NOUN      
          │┌─► are       auxpass   be       AUX       
┌┬┬┬──────┴┴── exposed   ROOT      expose   VERB      
│││└─►┌─────── to        prep      to       ADP       
│││   │  ┌───► a         det       a        DET       
│││   │  │┌──► Device    compound  device   NOUN      
│││   │  ││┌─► Lost      compound  lose     VERB      
│││   └─►└┴┴── Threat    pobj      threat   NOUN      
││└────►┌───── with      prep      with     ADP       
││      └─►┌── prob      pobj      prob     PROPN     
││         └─► 5.0       nummod    5.0      NUM       
│└───────────► and       cc        and      CCONJ     
└─────────►┌── impact    conj      impact   NOUN      
           └─► 1.0       dobj      1.0      NUM       


Creating necessary dictionaries

In [11]:
import functions
from functions import *

threats_file = '/content/threats.txt'
threats_dict = terms_dict(threats_file)




# Añadimos el termino 'intentional' a todos los términos que tengan la palabra 
# 'deliberated' porque 'deliberated' da problemas y sugerimos evitar esa palabra
for k,v in threats_dict.items():
  if 'Deliberated' in threats_dict[k]:
    add_word_to_dict(threats_dict,k, 'intentional')

# añadimos las palabras para el acrónimo HW
for k,v in threats_dict.items():
  if 'HW' in threats_dict[k]:
    add_word_to_dict(threats_dict,k, 'hardware')

# añadimos las palabras para el acrónimo SW
for k,v in threats_dict.items():
  if 'SW' in threats_dict[k]:
    add_word_to_dict(threats_dict,k, 'software')


assets_file = '/content/assets.txt'
assets_dict = terms_dict(assets_file)



In [7]:
def process_asset_threat_impact_prob_sentence(sentence: str, result: {}):
  """Function that process a sentences finding the terms:
  asset, threatType, prob and impact"""
  sentence = sentence.replace(",",".") # Para admitir decimales con "," o "."
  doc = nlp(str(sentence)) 
  
  asset = extract_passive_subject(sentence)      
  asset = clean(asset)
  asset_matches = search_matches(assets_dict, asset)
  asset_count = counter(asset_matches)
  asset = higher_frecuency_term(asset_count)
  result["asset"] = asset


  threat = extract_threat_passive(sentence)
  threat = clean(threat)
  threat_matches = search_matches(threats_dict, threat)  
  threat_count = counter(threat_matches)  
  threat = higher_frecuency_term(threat_count)  
  result["threatType"] = threat

  result["prob"] = extract_prob(sentence)
  result["impact"] = extract_impact(sentence)



In [10]:
result = {
    "asset": ""
}

sentence = 'Computers are exposed to a Device Lost Threat with prob four and impact 1.0.'


process_asset_threat_impact_prob_sentence(sentence,result)
result = {"result": [result]}
write_json("results.json", result) # escribir archivo con los resultados
result

{'result': [{'asset': 'Computer',
   'impact': 1,
   'prob': 4,
   'threatType': 'DeviceLost'}]}